In [ ]:
import pandas as pd
import sys

In [ ]:
!pip install python-dotenv

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')
sys.path.insert(0,'/content/gdrive/MyDrive/ScalableML/FinalProject')

from functions import *

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


###[Run the next cell]!!!!!!!!Reading air data set(pm10, pm...) from Github:

In [ ]:
url = 'https://github.com/reyrobs/Scalable_ML_Project_AQ/raw/main/Datasets/air_quality_df.csv'
df_air_quality = pd.read_csv(url)

df_air_quality.head()

###Reading air data set(pm10, pm25,...) from Google drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
url = '/content/gdrive/MyDrive/ScalableML/FinalProject/Shared_final_project_scalableML/air_quality_df.csv'
df_air_quality = pd.read_csv(url)

df_air_quality.head()

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


,date,pm25,pm10,o3,no2,so2,co,AQI
0,2022/12/1,78.0,37.0,4.0,18.0,0.0,0.0,78.0
1,2022/12/2,93.0,24.0,3.0,17.0,0.0,0.0,93.0
2,2022/12/3,68.0,30.0,5.0,17.0,0.0,0.0,68.0
3,2022/12/4,79.0,29.0,1.0,16.0,0.0,0.0,79.0
4,2022/12/5,77.0,34.0,0.0,17.0,0.0,0.0,77.0


In [ ]:
df_air_quality.date = df_air_quality.date.apply(timestamp_2_time_slash)
#df_air_quality.sort_values(by = ['city','date'],inplace = True,ignore_index = True)
df_air_quality.sort_values(by = ['date'],inplace = True,ignore_index = True)
df_air_quality.head()

,date,pm25,pm10,o3,no2,so2,co,AQI
0,1388448000000,0.0,19.0,22.0,30.0,0.0,2.0,30.0
1,1388534400000,43.0,30.0,22.0,42.0,0.0,4.0,43.0
2,1388620800000,53.0,33.0,22.0,34.0,0.0,2.0,53.0
3,1388707200000,50.0,28.0,23.0,41.0,0.0,3.0,50.0
4,1388793600000,52.0,23.0,19.0,34.0,0.0,3.0,52.0


###!!!!!!!!!Reading weather data(wind direction)  from Github 

In [ ]:
url = 'https://github.com/reyrobs/Scalable_ML_Project_AQ/raw/main/Datasets/visual_crossing_df.csv'
df_weather = pd.read_csv(url)

df_weather.head(3)

###Reading weather data (wind direction) from Google drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
url = '/content/gdrive/MyDrive/ScalableML/FinalProject/Shared_final_project_scalableML/visual_crossing_df.csv'
df_weather = pd.read_csv(url)

df_weather.head()

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


,date,winddir
0,2022-01-01,178.8
1,2022-01-02,221.0
2,2022-01-03,224.0
3,2022-01-04,280.8
4,2022-01-05,291.2


In [ ]:
df_weather.shape

(354, 2)

In [ ]:
df_weather.date = df_weather.date.apply(timestamp_2_time_dash)
df_weather.sort_values(by=['date'],inplace=True, ignore_index=True)
df_weather.head(3)

,date,winddir
0,1640995200000,178.8
1,1641081600000,221.0
2,1641168000000,224.0


In [ ]:
!pip install -U hopsworks 


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#!pip install pandas
import hopsworks

In [ ]:
project = hopsworks.login()

fs = project.get_feature_store() 

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/5315
Connected. Call `.close()` to terminate connection gracefully.


In [ ]:
air_quality_fg = fs.get_or_create_feature_group(
        name = 'air_quality_fg',
        description = 'Air Quality characteristics of each day',
        version = 2,
        primary_key = ['date'],# ['city','date'],
        online_enabled = True,
        event_time = 'date'
    )    
air_quality_fg.insert(df_air_quality)


Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/5315/fs/5235/fg/8663


Uploading Dataframe: 0.00% |          | Rows 0/3146 | Elapsed Time: 00:00 | Remaining Time: ?

Launching offline feature group backfill job...
Backfill Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/5315/jobs/named/air_quality_fg_2_offline_fg_backfill/executions


(<hsfs.core.job.Job at 0x7f1705b24220>, None)

In [ ]:
weather_fg = fs.get_or_create_feature_group(
        name = 'weather_fg',
        description = 'Weather characteristics of each day',
        version = 2,
        primary_key = ['city','date'],
        online_enabled = True,
        event_time = 'date'
    )    

weather_fg.insert(df_weather)

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/5315/fs/5235/fg/8664


Uploading Dataframe: 0.00% |          | Rows 0/354 | Elapsed Time: 00:00 | Remaining Time: ?

Launching offline feature group backfill job...
Backfill Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/5315/jobs/named/weather_fg_2_offline_fg_backfill/executions


(<hsfs.core.job.Job at 0x7f1705b8d2b0>, None)